<a href="https://colab.research.google.com/github/MRX418/2010/blob/main/accuracy.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [62]:
pip install pydicom

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [61]:
pip install xlsxwriter

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [63]:

import pandas as pd
import cv2
import pydicom
import numpy as np
import matplotlib.pyplot as plt
from skimage import exposure, measure
from scipy import ndimage
import os
import xlsxwriter
from google.colab import output

workbook = xlsxwriter.Workbook('accuracy101.xlsx')
worksheet = workbook.add_worksheet()
worksheet.write('A1', 'image')
worksheet.write('B1', 'accuracy')
f = os.listdir("sample_data/image")
images=[]
for image in f:
  name = image.split(".")
  if name[1] == 'dcm':
    images.append(image)
def algo(file):
  file1= images[file]

  images1= pydicom.read_file(file1);
  img_data1 = images1.pixel_array

  fig2 = plt.figure()
  plt.imshow(img_data1, cmap="gray")


  y = (img_data1<=-1000)*-1000 + (img_data1>=400)*400 + ((400>img_data1) & (img_data1>-1000)) * img_data1
  norm = cv2.normalize(y, None, alpha=0, beta=1, norm_type=cv2.NORM_MINMAX, dtype=cv2.CV_32F)
  th = norm<.95
  th = np.double(th*255)
  kernel1 = np.ones((3,3), np.uint8)
  th = cv2.erode(th, kernel1, iterations=1)
  all_labels = measure.label(th)
  th[all_labels==all_labels[0,0]]=0
  th[all_labels==all_labels[-1,-1]]=0
  th1 = ndimage.binary_fill_holes(th,  structure=np.ones((3,3))).astype(int)
  all_labels = measure.label(th1)

  for i in range(np.max(all_labels)+1):
    k = np.sum(all_labels==i)
    if k<500:
      th1[all_labels==i]=0

  kernel = np.ones((30,30), np.uint8)
  kernel1 = np.ones((7,7), np.uint8)
  th2 = np.double(th1*255)
  img_dilation = cv2.dilate(th2, kernel, iterations=1)
  img_erosion = cv2.erode(img_dilation, kernel1, iterations=2)


  fig2 = plt.figure()
  plt.imshow((img_erosion>0)*img_data1, cmap="gray")
  plt.pause(.001)
  return

row = 1
column = 1
for file in range(len(images)):
  algo(file);
  worksheet.write(row, column, input('Enter accuracy:'))
  worksheet.write(row, column-1, images[file])
  row += 1
  #os.system('cls')
  output.clear()
 
# Finally, close the Excel file
# via the close() method.
workbook.close()



In [ ]:
from google.colab import drive
drive.mount('/content/drive')